
## Small foward backward example

In [1]:
#Only once when launching notebook
import os
import sys
print(sys.path)
sys.path.append('C:\\Users\\Zeta\\Documents\\acou_sommeil_HD_ENS\\Tinnitus-n-Sleep')

import os
PATH = os.getcwd() 
import sys
sys.path.append(PATH + '/../')

['C:\\Users\\Zeta\\Documents\\acou_sommeil_HD_ENS\\provisoire', 'C:\\Users\\Zeta\\Anaconda3\\python37.zip', 'C:\\Users\\Zeta\\Anaconda3\\DLLs', 'C:\\Users\\Zeta\\Anaconda3\\lib', 'C:\\Users\\Zeta\\Anaconda3', '', 'C:\\Users\\Zeta\\Anaconda3\\lib\\site-packages', 'C:\\Users\\Zeta\\Anaconda3\\lib\\site-packages\\win32', 'C:\\Users\\Zeta\\Anaconda3\\lib\\site-packages\\win32\\lib', 'C:\\Users\\Zeta\\Anaconda3\\lib\\site-packages\\Pythonwin', 'C:\\Users\\Zeta\\Anaconda3\\lib\\site-packages\\IPython\\extensions', 'C:\\Users\\Zeta\\.ipython']


In [3]:
import matplotlib.pyplot as plt
%matplotlib qt
import numpy as np
import mne
import scipy
import seaborn as sns
from tinnsleep.config import Config
from tinnsleep.data import CreateRaw, RawToEpochs_sliding, CleanAnnotations, AnnotateRaw_sliding
from tinnsleep.classification import AmplitudeThresholding
from tinnsleep.check_impedance import create_annotation_mne, Impedance_thresholding_sliding, check_RMS, fuse_with_classif_result
from tinnsleep.signal import rms
from tinnsleep.visualization import plotTimeSeries, plotAnnotations, zoom_effect
from IPython.core.display import display
from ipywidgets import widgets
print("Config loaded")


Config loaded


In [4]:
print(Config.bruxisme_files[44:46])
print("")
ME_files=[Config.bruxisme_files[5], Config.bruxisme_files[9], 
          Config.bruxisme_files[14], Config.bruxisme_files[22]]
ME_files.extend(Config.bruxisme_files[28:44])
ME_files.extend(Config.bruxisme_files[46:])

print(ME_files)
print(len(ME_files))

['E:/Acou_sommeil/EDF_V2_PAUL\\robin_nuit_23_sept.edf', 'E:/Acou_sommeil/EDF_V2_PAUL\\robin_nuit_son_24_sept.edf']

['E:/Acou_sommeil/EDF_V2_PAUL\\1DA15_nuit_son.edf', 'E:/Acou_sommeil/EDF_V2_PAUL\\1GB19_nuit_hab.edf', 'E:/Acou_sommeil/EDF_V2_PAUL\\1MF19_nuit_hab.edf', 'E:/Acou_sommeil/EDF_V2_PAUL\\1RA17_nuit_hab.edf', 'E:/Acou_sommeil/EDF_V2_PAUL\\HZB_nuit_1.edf', 'E:/Acou_sommeil/EDF_V2_PAUL\\HZB_nuit_2.edf', 'E:/Acou_sommeil/EDF_V2_PAUL\\HZB_nuit_3.edf', 'E:/Acou_sommeil/EDF_V2_PAUL\\SCHMIDTLIN_nuit_1_dec_OD__0to0.edf', 'E:/Acou_sommeil/EDF_V2_PAUL\\SCHMIDTLIN_nuit_3_dec_OD__4to0to2.edf', 'E:/Acou_sommeil/EDF_V2_PAUL\\SCHMIDTLIN_nuit_4_dec_OD__3to3.edf', 'E:/Acou_sommeil/EDF_V2_PAUL\\SCHMIDTLIN_nuit_5_dec_OD__0to1.edf', 'E:/Acou_sommeil/EDF_V2_PAUL\\SCHM_nuit_1.edf', 'E:/Acou_sommeil/EDF_V2_PAUL\\SCHM_nuit_2.edf', 'E:/Acou_sommeil/EDF_V2_PAUL\\SCHM_nuit_3.edf', 'E:/Acou_sommeil/EDF_V2_PAUL\\Schmidtlin_nuit_2_dec_3to0to4.edf', 'E:/Acou_sommeil/EDF_V2_PAUL\\Unger_2.edf', 'E:/Acou_somm

In [5]:
filenames = ME_files[:6]  # load file from config
#filenames=bad_baseline


for filename in filenames:
    picks_chan = ['Airflow']           # middle ear electrodes

    raw  = mne.io.read_raw_edf(filename, preload=False)  # prepare loading
    raw  = CreateRaw(raw[picks_chan][0], raw.info["sfreq"] picks_chan, ch_types=['emg'])        # pick channels and load

    ch_names = raw.info["ch_names"]
    print("Data filtered")
    
    #epoching
    sfreq = raw.info["sfreq"]
    window_length = 1                    # in seconds
    duration = int(window_length * sfreq)   # in samples
    interval = duration                     # no overlapping
    epochs = RawToEpochs_sliding(raw, duration=duration, interval=interval)
    print(f"Epochs done, shape {epochs.shape}")
    
    
    #Foward
    # compute the sum of power over electrodes and samples in each window
    pipeline = AmplitudeThresholding(abs_threshold=0., rel_threshold=3.5, n_adaptive=60)
    X        = rms(epochs) # take only valid labels
    labels   = pipeline.fit_predict(X)
    print(f"bursts count: {np.sum(labels)}/{len(labels)} ({np.sum(labels) / len(labels) * 100:.2f}%)")
    print(f"bursts time: {np.sum(labels) * window_length} seconds")
    
    #Adding annotations
    raw = CleanAnnotations(raw)
    dict_annotations = {2: "tot"}
    raw = AnnotateRaw_sliding(raw, labels, 
                    dict_annotations=dict_annotations, duration=duration, interval=interval)

    #Backward
    #Reversing epochs array
    epochs = epochs[::-1]
    
     # compute the sum of power over electrodes and samples in each window
    pipeline = AmplitudeThresholding(abs_threshold=0., rel_threshold=3.5, n_adaptive=60)
    X        = rms(epochs) # take only valid labels
    labels   = pipeline.fit_predict(X)
    #Reversing labels
    labels = labels[::-1]
    

    print(f"bursts count: {np.sum(labels)}/{len(labels)} ({np.sum(labels) / len(labels) * 100:.2f}%)")
    print(f"bursts time: {np.sum(labels) * window_length} seconds")
    
    #raw = CleanAnnotations(raw)
    dict_annotations = {2: "tot"}
    raw = AnnotateRaw_sliding(raw, labels, 
                    dict_annotations=dict_annotations, duration=duration, interval=interval)
    print("Raw annotated")
    scalings = dict(emg=1)
    raw.plot(scalings = scalings)
    plt.title(filename)

Extracting EDF parameters from E:\Acou_sommeil\EDF_V2_PAUL\1DA15_nuit_son.edf...
EDF file detected
Setting channel info structure...
Creating raw.info structure...


<ipython-input-5-a6b76c3aae4d>:8: RuntimeWarning: 6 channel names are too long, have been truncated to 15 characters:
['Inductance Abdom', 'Inductance Thora', 'Jambe droite Imp', 'Jambe gauche Imp', 'Tension (aliment', 'Tension (Bluetoo']
  raw  = mne.io.read_raw_edf(filename, preload=False)  # prepare loading


Data filtered
Epochs done, shape (32460, 1, 200)
bursts count: 122/32460 (0.38%)
bursts time: 122 seconds
bursts count: 129/32460 (0.40%)
bursts time: 129 seconds
Raw annotated
Extracting EDF parameters from E:\Acou_sommeil\EDF_V2_PAUL\1GB19_nuit_hab.edf...
EDF file detected
Setting channel info structure...
Creating raw.info structure...


<ipython-input-5-a6b76c3aae4d>:8: RuntimeWarning: 6 channel names are too long, have been truncated to 15 characters:
['Inductance Abdom', 'Inductance Thora', 'Jambe droite Imp', 'Jambe gauche Imp', 'Tension (aliment', 'Tension (Bluetoo']
  raw  = mne.io.read_raw_edf(filename, preload=False)  # prepare loading


KeyboardInterrupt: 